In [1]:
%matplotlib inline

import numpy as np
from pandas import DataFrame
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers, models, initializers
from tensorflow.keras.applications import VGG16

np.random.seed(20210101)
tf.random.set_seed(20210101)

print("tensorflow version : " + tf.__version__)

tensorflow version : 2.3.1


In [2]:
model = VGG16(weights='imagenet')

553467904/553467096 [==============================] - 17s 0us/step


In [3]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions

img_path = '.\\data\\creative_commons_elephant.jpg'
#imgは224*224のPIL画像
img = image.load_img(img_path, target_size=(224, 224))
#(224, 224, 3)のfloat32 numpy配列に変換
x = image.img_to_array(img)
#入力層に合わせて(1, 224, 224, 3)に変換
x = np.expand_dims(x, axis=0)
#各チャンネルごとに色を正規化
x = preprocess_input(x)

In [4]:
preds = model.predict(x)
print('Predicted:', decode_predictions(preds, top=3)[0])

40960/35363 [==================================] - 0s 0us/step
Predicted: [('n02504458', 'African_elephant', 0.9098858), ('n01871265', 'tusker', 0.08572497), ('n02504013', 'Indian_elephant', 0.0043471376)]


In [5]:
np.argmax(preds[0])

386

In [9]:
from tensorflow.keras import backend as K

african_elephant_output = model.output[:, 386]

last_conv_layer = model.get_layer('block5_conv3')

grads = K.gradients(african_elephant_output, last_conv_layer.output)[0]

pooled_grads = K.mean(grads, axis=(0, 1, 2))

iterate = K.function([model.input], [pooled_grads, last_conv_layer.output[0]])

pooled_grads_value, conv_layer_output_value = iterate([x])

for i in range(512):
    conv_layer_output_value[:, :, i] *= pooled_grads_value[i]

heatmap = np.mean(conv_layer_output_value, axis=-1)

RuntimeError: tf.gradients is not supported when eager execution is enabled. Use tf.GradientTape instead.

In [ ]:
heatmap = np.maximum(heatmap, 0)
heatmap /= np.max(heatmap)
plt.matshow(heatmap)
plt.show()

In [ ]:
import cv2

# We use cv2 to load the original image
img = cv2.imread(img_path)

# We resize the heatmap to have the same size as the original image
heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))

# We convert the heatmap to RGB
heatmap = np.uint8(255 * heatmap)

# We apply the heatmap to the original image
heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

# 0.4 here is a heatmap intensity factor
superimposed_img = heatmap * 0.4 + img

# Save the image to disk
cv2.imwrite('.\\data\\elephant_cam.jpg', superimposed_img)